In [1]:
import sys 
sys.path.append('../scripts/')
from robot import *
from scipy.stats import multivariate_normal
import random #追加
import copy

In [2]:
class Particle:
    def __init__(self, init_pose, weight):
        if init_pose is None:
            self.pose = np.array([
                np.random.uniform(-5.0, 5.0),
                np.random.uniform(-5.0, 5.0),
                np.random.uniform(-math.pi, math.pi)]
            ).T
        else:
            self.pose = init_pose
            
        self.weight = weight
        
    def motion_update(self, nu, omega, time, noise_rate_pdf): 
        ns = noise_rate_pdf.rvs()
        pnu = nu + ns[0]*math.sqrt(abs(nu)/time) + ns[1]*math.sqrt(abs(omega)/time)
        pomega = omega + ns[2]*math.sqrt(abs(nu)/time) + ns[3]*math.sqrt(abs(omega)/time)
        self.pose = IdealRobot.state_transition(pnu, pomega, time, self.pose)
        
    def observation_update(self, observation, envmap, distance_dev_rate, direction_dev):  #変更
        for d in observation:
            obs_pos = d[0]
            obs_id = d[1]
            
            ##パーティクルの位置と地図からランドマークの距離と方角を算出##
            pos_on_map = envmap.landmarks[obs_id].pos
            particle_suggest_pos = IdealCamera.relative_polar_pos(self.pose, pos_on_map)
            
            ##尤度の計算##
            distance_dev = distance_dev_rate*particle_suggest_pos[0]
            cov = np.diag(np.array([distance_dev**2, direction_dev**2]))
            self.weight *= multivariate_normal(mean=particle_suggest_pos, cov=cov).pdf(obs_pos)

In [3]:
class Mcl:  
    def __init__(self, envmap, init_pose, num, motion_noise_stds, \
                 distance_dev_rate=0.14, direction_dev=0.05):
        self.particles = [Particle(init_pose, 1.0/num) for i in range(num)]
        self.map = envmap
        self.distance_dev_rate = distance_dev_rate
        self.direction_dev = direction_dev

        v = motion_noise_stds
        c = np.diag([v["nn"]**2, v["no"]**2, v["on"]**2, v["oo"]**2])
        self.motion_noise_rate_pdf = multivariate_normal(cov=c)
        
        self.ml = self.particles[0] #追加
        
    def set_ml(self): #追加
        i = np.argmax([p.weight for p in self.particles])
        self.ml = self.particles[i]
        
    def motion_update(self, nu, omega, time): 
        for p in self.particles: p.motion_update(nu, omega, time, self.motion_noise_rate_pdf)
            
    def observation_update(self, observation): 
        for p in self.particles:
            p.observation_update(observation, self.map, self.distance_dev_rate, self.direction_dev) 
        self.set_ml() #リサンプリング前に実行
        self.resampling() 
            
    def resampling(self): 
        ws = [e.weight for e in self.particles]    # 重みのリストを作る
        
        #重みの和がゼロに丸め込まれるとサンプリングできなくなるので小さな数を足しておく
        if sum(ws) < 1e-100: ws = [e + 1e-100 for e in ws]
        
        # パーティクルのリストから、weightsのリストの重みに比例した確率で、num個選ぶ    
        ps = random.choices(self.particles, weights=ws, k=len(self.particles))  
        
        # 選んだリストからパーティクルを取り出し、重みを均一に
        self.particles = [copy.deepcopy(e) for e in ps]
        for p in self.particles: p.weight = 1.0/len(self.particles)
        
    def draw(self, ax, elems):  
        xs = [p.pose[0] for p in self.particles]
        ys = [p.pose[1] for p in self.particles]
        vxs = [math.cos(p.pose[2])*p.weight*len(self.particles) for p in self.particles] #重みを要素に反映
        vys = [math.sin(p.pose[2])*p.weight*len(self.particles) for p in self.particles]  #重みを要素に反映
        elems.append(ax.quiver(xs, ys, vxs, vys, \
                               angles='xy', scale_units='xy', scale=1.5, color="blue", alpha=0.5)) #変更

In [4]:
class MclAgent(Agent): 
    def __init__(self, time_interval, nu, omega, particle_pose, envmap, particle_num=100, \
                motion_noise_stds={"nn":0.19, "no":0.001, "on":0.13, "oo":0.2}): #2行目にenvmapを追加
        super().__init__(nu, omega)
        self.pf = Mcl(envmap, particle_pose, particle_num, motion_noise_stds) #envmapを追加
        self.time_interval = time_interval
        
        self.prev_nu = 0.0
        self.prev_omega = 0.0
        
    def decision(self, observation=None): 
        self.pf.motion_update(self.prev_nu, self.prev_omega, self.time_interval)
        self.prev_nu, self.prev_omega = self.nu, self.omega
        self.pf.observation_update(observation)
        return self.nu, self.omega
        
    def draw(self, ax, elems):
        self.pf.draw(ax, elems)
        x, y, t = self.pf.ml.pose #以下追加
        s = "({:.2f}, {:.2f}, {})".format(x,y,int(t*180/math.pi)%360)
        elems.append(ax.text(x, y+0.1, s, fontsize=8))

In [5]:
def trial(animation): ###mclkidnap1test
    time_interval = 0.1
    world = World(30, time_interval, debug=not animation) 

    ## 地図を生成して3つランドマークを追加 ##
    m = Map()                                  
    m.append_landmark(Landmark(-4,2))
    m.append_landmark(Landmark(2,-3))
    m.append_landmark(Landmark(3,3))
    world.append(m)

    ## ロボットとパーティクルの初期姿勢を選ぶ ##
  #  particle_pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0),np.random.uniform(-math.pi, math.pi)]).T
    robot_pose = np.array([np.random.uniform(-5.0, 5.0), np.random.uniform(-5.0, 5.0), np.random.uniform(-math.pi, math.pi)]).T
        
#    a = MclAgent(time_interval, 0.2, 10.0/180*math.pi, particle_pose, m, particle_num=100) #初期姿勢をparticle_poseに
    a = MclAgent(time_interval, 0.2, 10.0/180*math.pi, robot_pose, m, particle_num=100) #初期姿勢をparticle_poseに
    r = Robot(robot_pose, sensor=Camera(m), agent=a, color="red") #初期姿勢をrobot_poseに
    world.append(r)

    world.draw()
    return (r.pose, a.pf.ml.pose)

In [6]:
ok = 0 
for i in range(1000):
    actual, estm = trial(False)
    diff = math.sqrt((actual[0]-estm[0])**2 + (actual[1]-estm[1])**2)
    print(i, "真値:", actual, "推定値", estm, "誤差:", diff)
    if diff <= 1.0:
        ok += 1

ok

0 真値: [ 1.91952893 -5.38803332  7.25415928] 推定値 [ 1.93319368 -5.16633271  7.15331611] 誤差: 0.22212133344857804
1 真値: [-0.14837538  3.44669871  6.83516208] 推定値 [-0.05729652  3.81612411  6.73899978] 誤差: 0.380487171831744
2 真値: [ 4.30633506 -2.17493802  2.1608802 ] 推定値 [ 3.55846509 -4.11991699  2.03580894] 誤差: 2.083807254103387
3 真値: [ 0.02233594 -5.03466298  2.45696183] 推定値 [ 0.21428266 -4.07383072  2.35309473] 誤差: 0.9798174228202109
4 真値: [-0.75060789  2.04861305  4.59650928] 推定値 [-0.68897393  2.41021938  4.56165472] 誤差: 0.3668213213923223
5 真値: [-2.55165814 -2.06611717  3.98733469] 推定値 [-2.24279636 -2.52759756  3.68379713] 誤差: 0.5553014969488148
6 真値: [-1.08211307  1.8835021   2.43498009] 推定値 [-1.20676518  1.85499752  2.46451576] 誤差: 0.1278696950853028
7 真値: [-4.58218659  0.20260863  6.17838509] 推定値 [-6.84196499  1.02372305  4.73638071] 誤差: 2.404335111935143
8 真値: [0.13351639 3.10142146 3.50812294] 推定値 [0.31415509 2.94481315 3.41243707] 誤差: 0.23907425934092588
9 真値: [ 1.85077909 -1.9851

/usr/local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


20 真値: [-2.79300196  0.68630962  2.61728146] 推定値 [-2.35138966  1.15913622  2.98398076] 誤差: 0.6469825493451582
21 真値: [-2.17606731 -4.61371049  3.10988476] 推定値 [-1.31658065 -4.03841766  2.81985642] 誤差: 1.0342529460052425
22 真値: [ 2.54354073 -1.16687749  4.33807619] 推定値 [ 1.94905589 -1.28374273  4.62649584] 誤差: 0.6058627790098177
23 真値: [-5.4800129   2.779896    6.39581997] 推定値 [-5.17669537  2.97680201  6.16704738] 誤差: 0.3616261838545787
24 真値: [ 0.83115194 -0.34071483  2.73036168] 推定値 [ 0.79710121 -0.62347107  2.74917636] 誤差: 0.28479912961760295
25 真値: [-1.86968266 -0.22808454  7.40839065] 推定値 [-1.46781902  0.23321571  7.33115054] 誤差: 0.611794334291114
26 真値: [ 0.66895123 -2.52393002  3.25892343] 推定値 [ 0.56751654 -2.10892401  3.16345949] 誤差: 0.4272224069829713
27 真値: [-0.62013143  5.62800225  3.39364775] 推定値 [-0.05227665  6.02795182  3.38775103] 誤差: 0.69456367752909
28 真値: [-5.34828058  4.63115241  5.55937041] 推定値 [-5.0447988   4.61350485  5.44126906] 誤差: 0.3039944524082649
29 真値: [-5.7

96 真値: [1.19519892 3.50001645 6.43625607] 推定値 [1.49637872 3.01232401 6.68599437] 誤差: 0.5731955855610018
97 真値: [ 1.00759177 -3.38904586  5.42558359] 推定値 [ 0.54076734 -3.18000699  4.91968713] 誤差: 0.5114902769643281
98 真値: [-1.38924261 -6.34399667  6.72213538] 推定値 [-1.39114405 -5.7643502   6.64089981] 誤差: 0.5796495950067356
99 真値: [ 4.45043903 -2.04795258  3.69457325] 推定値 [ 3.9636699  -2.0156658   3.74329552] 誤差: 0.48783871777114884
100 真値: [1.01027927 1.56619604 5.54277012] 推定値 [1.35628665 1.88571746 5.47821384] 誤差: 0.47097244601078675
101 真値: [-0.72437553  1.71314797  6.44363072] 推定値 [-0.87665815  1.47656338  6.55903145] 誤差: 0.28135789380533366
102 真値: [-2.7331868 -0.9455719  3.832163 ] 推定値 [-2.98430701 -1.19807779  3.65007365] 誤差: 0.3561187748742077
103 真値: [0.1934378  4.57856709 3.485906  ] 推定値 [0.32055198 4.40801995 3.4809139 ] 誤差: 0.21270717478209758
104 真値: [ 2.12117877 -1.14014823  3.04384313] 推定値 [ 2.404106   -0.85759904  2.75305777] 誤差: 0.399852296025933
105 真値: [-2.90580258  3

171 真値: [ 0.21816162 -2.26477856  6.12738468] 推定値 [ 0.09511424 -2.108848    6.0009486 ] 誤差: 0.19863282397400853
172 真値: [-4.62629719 -1.29127419  4.81937804] 推定値 [-5.07977506 -1.0418004   4.24776245] 誤差: 0.5175706227577974
173 真値: [ 6.38033013 -3.63479924  2.04887422] 推定値 [ 4.23350672 -2.64726036  2.95813083] 誤差: 2.36306660622296
174 真値: [-2.26342026  1.41666627  3.96269741] 推定値 [-2.34539532  0.79840574  3.7499993 ] 誤差: 0.6236713845242857
175 真値: [ 4.22599022 -0.08095263  3.92279196] 推定値 [ 4.70666754 -0.15589915  3.7675745 ] 誤差: 0.486485010300953
176 真値: [2.24988056 0.30897792 8.52053507] 推定値 [2.3662404  0.3146563  8.51116646] 誤差: 0.11649831579180571
177 真値: [3.89270739 5.70055643 3.2991995 ] 推定値 [3.62516196 5.39279952 3.36172923] 誤差: 0.4077926817086897
178 真値: [-1.22042043  0.58152179  2.46323628] 推定値 [-1.07509245  0.59498019  2.4187393 ] 誤差: 0.14594982799686101
179 真値: [ 3.23904526 -2.39170916  3.6389391 ] 推定値 [ 3.28992773 -2.43671886  3.5819925 ] 誤差: 0.06793304472618182
180 真値: [-4.

...

771 真値: [0.46079286 0.39781728 4.75620077] 推定値 [1.30686637 0.83088352 4.50092512] 誤差: 0.9504665922237752
772 真値: [ 3.06615761 -0.76341436  3.2301765 ] 推定値 [ 3.49576736 -0.57056953  3.44255551] 誤差: 0.47090728227835416
773 真値: [-1.48043001 -3.08945207  5.15584179] 推定値 [-1.62543806 -3.10312953  5.4389327 ] 誤差: 0.14565166500652665
774 真値: [2.58741397 3.29473093 2.2537164 ] 推定値 [2.59427233 3.35617203 1.91834812] 誤差: 0.0618226993574901
775 真値: [-0.40778193 -4.26487487  4.90833734] 推定値 [-1.40703788 -4.09956468  4.94984129] 誤差: 1.0128375518613006
776 真値: [2.73091201 6.00117979 4.26158424] 推定値 [2.44536819 5.80338805 4.40627144] 誤差: 0.34735693225457037
777 真値: [ 4.18694921 -0.70863735  3.7556685 ] 推定値 [ 4.6078609  -0.67302794  3.66410405] 誤差: 0.42241529376524983
778 真値: [-2.99948626 -5.24405856  6.85978108] 推定値 [-3.09994407 -5.42312217  6.83786342] 誤差: 0.2053181584389497
779 真値: [-0.08365311 -4.10676828  6.01456236] 推定値 [ 0.19464186 -4.62227237  6.28973241] 誤差: 0.5858263922541722
780 真値: [-2.261

846 真値: [-2.2456795  -1.54696893  6.10995283] 推定値 [-2.1912603  -1.19041507  5.9942696 ] 誤差: 0.3606828373757241
847 真値: [0.71832083 4.00989426 7.22250056] 推定値 [0.68939676 4.07330609 7.09745704] 誤差: 0.06969692335458505
848 真値: [-5.92930448  4.2609629   4.79296069] 推定値 [-5.7735993   4.26192493  4.79473332] 誤差: 0.15570815277958805
849 真値: [-2.88171973  0.19274104  6.88605813] 推定値 [-3.80723181 -0.484511    7.01341064] 誤差: 1.1468404167370196
850 真値: [ 4.53326231 -0.97378023  7.15448399] 推定値 [ 4.28092136 -1.46636877  6.58163197] 誤差: 0.5534613124591261
851 真値: [-3.91075471 -4.15676452  3.38968704] 推定値 [-3.27972878 -3.11907448  3.02852326] 誤差: 1.2144934476780456
852 真値: [0.67963747 0.78472797 4.69886487] 推定値 [0.54155076 0.58229228 4.772443  ] 誤差: 0.24504723111260943
853 真値: [ 2.29389134 -3.98972345  8.42844981] 推定値 [ 2.48456014 -3.67005323  8.65324335] 誤差: 0.3722145029769168
854 真値: [-0.7874384  -2.2629738   7.60680664] 推定値 [-0.57112446 -1.68952372  7.74160105] 誤差: 0.6128920922391407
855 真値: [3

921 真値: [1.07748589 3.08406382 6.38648458] 推定値 [0.96985836 2.92876934 6.38842858] 誤差: 0.18894459886467135
922 真値: [ 5.36935763 -2.56317584  2.87321655] 推定値 [ 5.35308434 -2.78383261  2.78395287] 誤差: 0.22125602637828426
923 真値: [0.97740195 2.49930126 2.51541576] 推定値 [1.24317628 2.70001858 2.49148851] 誤差: 0.33305170731755057
924 真値: [-0.18546033 -3.42711969  6.92621768] 推定値 [-0.30333977 -3.30601429  6.79019821] 誤差: 0.16900318906224054
925 真値: [1.44307081 5.11729932 2.77634003] 推定値 [1.61891155 5.34715602 2.80519759] 誤差: 0.28940295488172346
926 真値: [ 1.79720526 -0.6867829   7.94911085] 推定値 [ 1.38013321 -0.67541099  7.75147161] 誤差: 0.4172270526859341
927 真値: [ 4.85354209 -0.40911478  7.96388736] 推定値 [ 4.70172882 -0.93845832  7.7810068 ] 誤差: 0.5506830810849259
928 真値: [3.55069435 3.49007993 6.92926306] 推定値 [3.31574301 3.43382884 6.6657614 ] 誤差: 0.24159122402628364
929 真値: [ 0.83933954 -0.97907679  6.71876148] 推定値 [ 0.67806875 -0.88815158  6.65270777] 誤差: 0.18513687458791575
930 真値: [-3.239668

996 真値: [-0.6787736   3.02400617  8.28026121] 推定値 [-0.9008829   3.10188667  8.22186156] 誤差: 0.23536761156465724
997 真値: [3.49302898 3.90027256 2.96313778] 推定値 [5.44138994 3.48265012 2.1160351 ] 誤差: 1.9926161051418765
998 真値: [-2.93385172 -3.14505989  5.32273505] 推定値 [-2.37503367 -3.00528858  5.2738615 ] 誤差: 0.576032658197147
999 真値: [-1.65814366 -0.82021714  8.3503697 ] 推定値 [-1.6386493  -1.1156079   8.31004437] 誤差: 0.2960333206073935


921